In [1]:
import pandas as pd
from tqdm import tqdm
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid

BASE_PATH = '../data/videos/'

# Create an instance of the speech segmenter
seg = Segmenter()

def segment_audio(video_id):
    # Path of the media
    media = BASE_PATH + f'{video_id}.mp4'
    # Starting audio segmentation
    segmentation = seg(media)
    return segmentation


# Data

In [2]:
# Load the video dataset
df_videos = pd.read_csv('../data/processed/parsed_dataset.csv', delimiter=';')
df_videos.drop_duplicates(subset=['id'], inplace=True)
df_videos.head()

,search_tag,video_path,video_classes,id,desc,create_time,is_duet_enabled,hashtags,cover,play_address,...,author_nickname,author_avatar,author_signature,author_is_verified,author_duet_setting,author_following_count,author_followers_count,author_heart_count,author_digg_count,author_heart
0,biden2020,data/videos/6892193566290889985.mp4,NaN,6892193566290889985,"We'll be right, thanks. New Zealand.. anyone? ...",1604713867,True,"bidenharris2020, straya, aussie, biden2020",https://p16-sign-sg.tiktokcdn.com/obj/tos-alis...,https://v16-webapp-prime.tiktok.com/video/tos/...,...,hangrybynature,NaN,NaN,NaN,NaN,278,1800000,82800000,7080,82800000
1,biden2020,data/videos/6839079433794505989.mp4,NaN,6839079433794505989,Reply to @nickh1940 #repost share this so more...,1592347270,True,", repost, trump, viral, fyp, foryoupage, parat...",https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,https://v16-webapp-prime.tiktok.com/video/tos/...,...,HD-80,NaN,NaN,NaN,NaN,163,208200,6600000,9305,6600000
2,biden2020,data/videos/6885403130297732357.mp4,NaN,6885403130297732357,anyways vote biden💙#fyp #foryou #foryoupage #b...,1603132900,False,"fyp, foryou, foryoupage, biden2020",https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,https://v16-webapp-prime.tiktok.com/video/tos/...,...,Tyler Bamberger,NaN,NaN,NaN,NaN,173,236200,15700000,100400,15700000
3,biden2020,data/videos/6889924077721521413.mp4,NaN,6889924077721521413,Oh my god Barack Obama 🥵🥵🥵 #biden2020,1604185495,True,biden2020,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,https://v16-webapp-prime.tiktok.com/video/tos/...,...,Meena Harris,NaN,NaN,NaN,NaN,277,2200000,81400000,14300,81400000
4,biden2020,data/videos/6891017029051108614.mp4,NaN,6891017029051108614,Reply to @taybarnett262 IDK HOW TO FEEL RN HE...,1604439965,True,", fyp, election2020, biden2020, trump2020",https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,https://v16-webapp-prime.tiktok.com/video/tos/...,...,jules,NaN,NaN,NaN,NaN,9971,206100,6200000,78000,6200000


# Audio segmentation

In [3]:
'''
Creates an instance of speech segmenter
this loads neural networks and may take a few seconds.
Warnings have no incience on the results.
'''
seg = Segmenter()

# Parsing videos to extract audio segmentation

In [10]:
import os
import sys

# Load the video dataset
df_videos = pd.read_csv('../data/processed/parsed_dataset.csv', delimiter=';')
df_videos.drop_duplicates(subset=['id'], inplace=True)

# Create a list to store parsed video segments
videos_segmentation_parsed = []

# Redirect the standard output to /dev/null
original_stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')

# Iterate over videos for audio segmentation
for video in tqdm(df_videos.loc[~df_videos['id'].isin(videos_segmentation_parsed), 'id'], ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}'):
    try:
        segmentation = segment_audio(video)
    except Exception as e:
        # Handle specific exceptions here if needed
        continue

    video_segmentations = {
        'video_id': video,
        'video_contains_music': False,
        'video_contains_male': False,
        'video_contains_female': False,
        'video_contains_noise': False,
        'video_contains_no_energy': False,
        'segments': []
    }

    for label, start, end in segmentation:
        if label == 'music':
            video_segmentations['video_contains_music'] = True
        elif label == 'male':
            video_segmentations['video_contains_male'] = True
        elif label == 'female':
            video_segmentations['video_contains_female'] = True
        elif label == 'noEnergy':
            video_segmentations['video_contains_no_energy'] = True
        elif label == 'noise':
            video_segmentations['video_contains_noise'] = True

        segmentation_dict = {
            'label': label,
            'start': start,
            'end': end
        }

        video_segmentations['segments'].append(segmentation_dict)

    videos_segmentation_parsed.append(video_segmentations)

# Restore the original standard output
sys.stdout.close()
sys.stdout = original_stdout


  0%|                                                                   | 0/2033c:\Users\snell\anaconda3\envs\thesis_env\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
c:\Users\snell\anaconda3\envs\thesis_env\lib\site-packages\pyannote\algorithms\utils\viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
  0%|                                                                   | 1/2033c:\Users\snell\anaconda3\envs\thesis_env\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such 

In [11]:
# Create DataFrame from video segmentations
df_videos_segmentation_parsed = pd.DataFrame(videos_segmentation_parsed)
df_videos_segmentation_parsed.to_csv('../data/processed/audio_segmentation_dataset.csv', index=False)

# Audio segment dataframe

In [12]:
# Create audio segment DataFrame
audio_segments_list = []
for video in videos_segmentation_parsed:
    for segment in video['segments']:
        video_segments_dict = {
            'video_id': video['video_id'],
            'label': segment['label'],
            'start': segment['start'],
            'end': segment['end']
        }
        audio_segments_list.append(video_segments_dict)

audio_segments_df = pd.DataFrame(audio_segments_list)
audio_segments_df.to_csv('../data/processed/video_audio_segments.csv', index=False)

# Saving dataset

In [13]:
# Merge video segmentations with the original video DataFrame
df_segmented_videos = df_videos.merge(df_videos_segmentation_parsed, left_on='id', right_on='video_id', how='left')
df_segmented_videos = df_segmented_videos.dropna(subset=['video_id'])
df_segmented_videos = df_segmented_videos.drop(['video_id'], axis=1)
df_segmented_videos.to_csv('../data/processed/videos_with_segmented_audio.csv', index=False)